#  Aperture photometry on MIRI stage-2 images

**Author**: Mattia Libralato, ESA/AURA Astronomer
<br>
**Last Updated**: July, 2021

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#imports)<br>
    2.2 [Plot configuration](#plot)<br>
    2.3 [Useful functions](#func)<br>
    2.4 [Setup PSF FWHM](#fwhm)<br>
    2.5 [Download data](#download)<br>
3. [A first glimpse to the MIRI data](#datalook)<br>
    3.1 [Data Quality (DQ) flags](#dqflag)<br>
4. [From the image to the astro-photometric catalog](#work)<br>
    4.1 [Sky background](#sky)<br>
    4.2 [Aperture photometry](#phot)<br>
    4.3 [From pixel to equatorial coordinates](#wcs)<br>
    4.4 [Photometric calibration](#cal)<br>
5. [Exercise](#exercise)<br>
6. [Bonus \#1: your first JWST mid-infrared color-magnitude diagram](#bonus1)<br>
7. [Bonus \#2: concentration indexes](#bonus2)<br>
8. [Exercise solutions](#exercisesol)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

In this notebook we provide an overview on aperture photometry with simulated images obtained with the Mid-Infrared Instrument ([MIRI](https://jwst-docs.stsci.edu/mid-infrared-instrument)) onboard the James Webb Space Telescope (JWST).

Images were simulated using [MIRISim](https://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/mirisim) and processed through the [JWST pipeline](https://jwst-pipeline.readthedocs.io/en/latest/). Each exposure was obtained with "FASTR1" readout, 20 groups per integration, and 3 integrations. Simulations were obtained using a [4-point dither pattern](https://jwst-docs.stsci.edu/mid-infrared-instrument/miri-operations/miri-dithering/miri-imaging-dithering#MIRIImagingDithering-4point) in F560W and F770W filters. The astronomical scene we will analyze is an homogeneous, mildly-crowded stellar field with a few background galaxies. The stellar population resambles that of a globular cluster; galaxies present random morphologies and fluxes. This mix of objects should provide a good benchmark for testing any photometric tools while waiting for real data.

In this notebook, we use *cal.fits* files. These images are the outputs of the [Stage 2](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_image2.html) pipeline. The Stage 2 pipeline takes in the slope images output from the [Stage 1](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_detector1.html) pipeline and applies instrumental corrections and calibrations.

At the end of this notebook, we should have learned (or refreshed our memory) on aperture photometry and on the MIRI imager.

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we import all necessary python packages, define some useful functions that will help us later during the analysis, read a table with the encircled-energy values for some MIRI filters, and finally download the simulated MIRI images.

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="imports"></a> ###

In [ ]:
#
# General tools
#
import glob
import os
import shutil
import sys
import random
import urllib
import zipfile
#
# Astropy tools
#
from astropy.coordinates import match_coordinates_sky, SkyCoord
from astropy.io import fits, ascii
from astropy.stats import SigmaClip, sigma_clipped_stats
from astropy.table import Table, Column, vstack
import astropy.units as u
from astropy.visualization import LogStretch, LinearStretch, PercentileInterval, ManualInterval
from astropy.nddata import Cutout2D
#
# JWST models
#
from jwst import datamodels, associations
from jwst.datamodels import ImageModel, dqflags
#
# Matplotlib tools
#
from matplotlib import style, pyplot as plt, rcParams
from matplotlib.colors import LogNorm
from matplotlib.pyplot import figure
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
#
# Numpy library
#
import numpy as np
#
# Photutils library and tools
#
import photutils
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils import Background2D, MedianBackground, ModeEstimatorBackground, MMMBackground
#
# Scipy tools
#
from scipy import stats
from scipy.interpolate import CubicSpline
#
# Use 90% of the window width
#
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### 2.2<font color='white'>-</font>Plot configuration<a class="anchor" id="plot"></a> ###

<div class="alert alert-block alert-warning">
    <h3><u><b>Warning</b></u></h3>

If the plots in this notebook don't render properly, you may need to install LaTeX. Find more information on a system-wide installation [here](https://www.latex-project.org/get/) and on a Jupyter-specific nbextension [here](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/latex_envs/README.html).
</div>

In [ ]:
%matplotlib widget
# Figure size
plt.rcParams['figure.figsize'] = [9, 9]
# Figure text and font
plt.rc('text', usetex=True)
plt.rc('font', size=15)

<p style="font-size:200%; color:#e56020; background-color:#1d1160;"><b><i>Reviewer note:</i> Begin PEP8 check cells (delete below when finished)</b></p>

In [ ]:
# disable all imported packages' loggers
import logging
logging.root.manager.loggerDict = {}

In [ ]:
# enable PEP8 checker for this notebook
%load_ext pycodestyle_magic
%flake8_on --ignore E261,E501,W291,W293

# only allow the checker to throw warnings when there's a violation
logging.getLogger('flake8').setLevel('ERROR')
logging.getLogger('stpipe').setLevel('ERROR')

<p style="font-size:200%; color:#e56020; background-color:#1d1160;"><b><i>Reviewer note:</i> End PEP8 check cells (delete above when finished)</b></p>

### 2.3<font color='white'>-</font>Useful functions<a class="anchor" id="func"></a> ###

Here you can find some convenient functions:

In [ ]:
def imshow_me_wcolorbar(img, vmin, vmax, tlabel, xlabel, ylabel, blabel, cmap):
    '''
    Function to show an image.
    
    Inputs
    ----------
    img : 2D numpy.ndarray
        The input 2D array.
    vmin : float
        The minimum value for the colorbar.
    vmax : float
        The maximum value for the colorbar.
    tlabel : string
        The plot title.
    xlabel : string
        The X-axis label.
    ylabel : string
        The Y-axis label.
    blabel : string
        The colorbar label.
    cmap : string
        The name of the colormap.
    '''
    
    fig, ax = plt.subplots()
    cax = ax.imshow(img, vmin=vmin, vmax=vmax, origin='lower', cmap=cmap)
    ax_divider = make_axes_locatable(ax)
    cax1 = ax_divider.append_axes('right', size='3%', pad='2%')
    cb = fig.colorbar(cax, cax=cax1)
    cb.ax.set_ylabel(r'{0}'.format(blabel))
    ax.set_xlabel(r'{0}'.format(xlabel))
    ax.set_ylabel(r'{0}'.format(ylabel))
    ax.set_title(r'{0}'.format(tlabel))
    plt.tight_layout()
    
    return

In [ ]:
def imshow_me_wcolorbar_setup(img, vmin, vmax, tlabel, xlabel, ylabel, blabel, cmap):
    '''
    Function to setup an image to show. Similar to imshow_me_wcolorbar,
    but it does not show the image directly, thus allowing to plot
    something else after the call.
    
    Inputs
    ----------
    img : 2D numpy.ndarray
        The input 2D array.
    vmin : float
        The minimum value for the colorbar.
    vmax : float
        The maximum value for the colorbar.
    tlabel : string
        The plot title.
    xlabel : string
        The X-axis label.
    ylabel : string
        The Y-axis label.
    blabel : string
        The colorbar label.
    cmap : string
        The name of the colormap.
    '''
    
    cax = ax.imshow(img, vmin=vmin, vmax=vmax, origin='lower', cmap=cmap)
    ax_divider = make_axes_locatable(ax)
    cax1 = ax_divider.append_axes('right', size='3%', pad='2%')
    cb = fig.colorbar(cax, cax=cax1)
    cb.ax.set_ylabel(r'{0}'.format(blabel))
    ax.set_xlabel(r'{0}'.format(xlabel))
    ax.set_ylabel(r'{0}'.format(ylabel))
    ax.set_title(r'{0}'.format(tlabel))
    
    return

In [ ]:
def imshow_cutouts(img, tab, id_sel, xlabel, ylabel, cmap):
    '''
    Function to show four cutouts from an image.
    
    Inputs
    ----------
    img : 2D numpy.ndarray
        The input 2D array.
    tab : Table
        The table with at least (x,y,id) columns.
    id_sel : array(int)
        The ID of the sources in tab to show in the cutouts.
    xlabel : string
        The X-axis label.
    ylabel : string
        The Y-axis label.
    cmap : string
        The name of the colormap.
    '''
    
    fig, ax = plt.subplots(2, 2)

    x, y = tab['x'][tab['id']==id_sel[0]], tab['y'][tab['id']==id_sel[0]]
    cutout = Cutout2D(img, (x, y), (51, 51))
    ax[0,0].imshow(cutout.data, vmin=-0.1, vmax=1.0, origin='lower', cmap=cmap)
    ax[0,0].scatter(cutout.input_position_cutout[0], cutout.input_position_cutout[1], lw=0.5, s=15, 
           marker='o', edgecolors='deepskyblue', facecolors='none')
    ax[0,0].annotate(r'(x,y)$\sim$({0},{1})'.format(int(x[0]),int(y[0])), xy=(0.5, 0.95), xycoords='axes fraction',
                    horizontalalignment='center', verticalalignment='center')
    ax[0,0].set_xlabel(r'{0}'.format(xlabel))
    ax[0,0].set_ylabel(r'{0}'.format(ylabel))

    x, y = tab['x'][tab['id']==id_sel[1]], tab['y'][tab['id']==id_sel[1]]
    cutout = Cutout2D(img, (x, y), (51, 51))
    ax[0,1].imshow(cutout.data, vmin=-0.1, vmax=1.0, origin='lower', cmap=cmap)
    ax[0,1].scatter(cutout.input_position_cutout[0], cutout.input_position_cutout[1], lw=0.5, s=15, 
           marker='o', edgecolors='deepskyblue', facecolors='none')
    ax[0,1].annotate(r'(x,y)$\sim$({0},{1})'.format(int(x[0]),int(y[0])), xy=(0.5, 0.95), xycoords='axes fraction',
                    horizontalalignment='center', verticalalignment='center')
    ax[0,1].set_xlabel(r'{0}'.format(xlabel))
    ax[0,1].set_ylabel(r'{0}'.format(ylabel))

    x, y = tab['x'][tab['id']==id_sel[2]], tab['y'][tab['id']==id_sel[2]]
    cutout = Cutout2D(img, (x, y), (51, 51))
    ax[1,0].imshow(cutout.data, vmin=-0.1, vmax=1.0, origin='lower', cmap=cmap)
    ax[1,0].scatter(cutout.input_position_cutout[0], cutout.input_position_cutout[1], lw=0.5, s=15, 
           marker='o', edgecolors='deepskyblue', facecolors='none')
    ax[1,0].annotate(r'(x,y)$\sim$({0},{1})'.format(int(x[0]),int(y[0])), xy=(0.5, 0.95), xycoords='axes fraction',
                    horizontalalignment='center', verticalalignment='center')
    ax[1,0].set_xlabel(r'{0}'.format(xlabel))
    ax[1,0].set_ylabel(r'{0}'.format(ylabel))

    x, y = tab['x'][tab['id']==id_sel[3]], tab['y'][tab['id']==id_sel[3]]
    cutout = Cutout2D(img, (x, y), (51, 51))
    ax[1,1].imshow(cutout.data, vmin=-0.1, vmax=1.0, origin='lower', cmap=cmap)
    ax[1,1].scatter(cutout.input_position_cutout[0], cutout.input_position_cutout[1], lw=0.5, s=15, 
           marker='o', edgecolors='deepskyblue', facecolors='none')
    ax[1,1].annotate(r'(x,y)$\sim$({0},{1})'.format(int(x[0]),int(y[0])), xy=(0.5, 0.95), xycoords='axes fraction',
                    horizontalalignment='center', verticalalignment='center')
    ax[1,1].set_xlabel(r'{0}'.format(xlabel))
    ax[1,1].set_ylabel(r'{0}'.format(ylabel))
    
    plt.tight_layout()
    
    return

2021-07-16 11:29:21,460 - stpipe - INFO - 23:30: E225 missing whitespace around operator
2021-07-16 11:29:21,461 - stpipe - INFO - 23:62: E225 missing whitespace around operator
2021-07-16 11:29:21,461 - stpipe - INFO - 25:9: E231 missing whitespace after ','
2021-07-16 11:29:21,462 - stpipe - INFO - 26:9: E231 missing whitespace after ','
2021-07-16 11:29:21,462 - stpipe - INFO - 27:12: E128 continuation line under-indented for visual indent
2021-07-16 11:29:21,463 - stpipe - INFO - 28:9: E231 missing whitespace after ','
2021-07-16 11:29:21,463 - stpipe - INFO - 28:62: E231 missing whitespace after ','
2021-07-16 11:29:21,463 - stpipe - INFO - 29:21: E128 continuation line under-indented for visual indent
2021-07-16 11:29:21,464 - stpipe - INFO - 30:9: E231 missing whitespace after ','
2021-07-16 11:29:21,464 - stpipe - INFO - 31:9: E231 missing whitespace after ','
2021-07-16 11:29:21,465 - stpipe - INFO - 33:30: E225 missing whitespace around operator
2021-07-16 11:29:21,465 - stpi

In [ ]:
def arcsec2pix(x):
    '''
    Function that converts arcsec in MIRIM pixels.
    It is used to setup the secondary axes in a plot.
    
    Inputs
    ----------
    x : float or array(float)
        The value in arcsec.
        
    Outputs
    ----------
    The value(s) in MIRIM pixel.
    '''
    
    return x/0.11

In [ ]:
def pix2arcsec(x):
    '''
    Function that converts MIRIM pixels in arcsec.
    It is used to setup the secondary axes in a plot.
    
    Inputs
    ----------
    x : float or array(float)
        The value in MIRIM pixels.
        
    Outputs
    ----------
    The value(s) in arcsec.
    '''
    
    return x*0.11

These last two functions will be used to speed-up the exercise.

In [ ]:
def prepare_image(img_name):
    '''
    Function that reads one image, flag all pixels with DQ flags different 
    from [0, 2, 4, 6] and adds a random background gradient.
    To be used in the exercise.
    
    Inputs
    ----------
    img_name : string
        The name of the image.
        
    Outputs
    ----------
    img : ImageModel
        The data model of the image.
    img_mod : 2D numpy.ndarray
        The image data with the additional random background gradient.
    '''
    
    img = datamodels.open(img_name)
    ok = np.zeros(img.data.shape, dtype='int')
    for v in [0, 2, 4, 6]:
        ok = ok + np.where(img.dq==v, 1, 0)

    img.data[ok==0] = np.nan
    
    x = np.linspace(0,1,img.meta.subarray.xsize)
    y = np.linspace(0,1,img.meta.subarray.ysize)
    X, Y = np.meshgrid(x, y)
    if (np.random.rand(1)>0.5):
        sx = +1
    else:
        sx = -1
    if (np.random.rand(1)>0.5):
        sy = +1
    else:
        sy = -1
    Z = (10*(np.random.rand(1)-0.5))*np.exp(sx*X/(4+2*(np.random.rand(1)-0.5)))*np.exp(sy*Y/(4+4*(np.random.rand(1)-0.5)))
    
    img_mod = img.data.copy() + Z
    
    _, sky_med, sky_sig = sigma_clipped_stats(img_mod[np.isfinite(img_mod)], sigma=5.0, maxiters=5)
    
    print('')
    print(r' Suggested min value: {0}'.format(sky_med-sky_sig))
    print(r' Suggested max value: {0}'.format(sky_med+sky_sig))
    print('')

    cmin = sky_med-sky_sig
    cmax = sky_med+sky_sig
    
    return img, img_mod, cmin, cmax

2021-07-16 11:29:26,034 - stpipe - INFO - 23:34: E225 missing whitespace around operator
2021-07-16 11:29:26,034 - stpipe - INFO - 25:16: E225 missing whitespace around operator
2021-07-16 11:29:26,034 - stpipe - INFO - 27:22: E231 missing whitespace after ','
2021-07-16 11:29:26,035 - stpipe - INFO - 27:24: E231 missing whitespace after ','
2021-07-16 11:29:26,036 - stpipe - INFO - 28:22: E231 missing whitespace after ','
2021-07-16 11:29:26,036 - stpipe - INFO - 28:24: E231 missing whitespace after ','
2021-07-16 11:29:26,036 - stpipe - INFO - 30:26: E225 missing whitespace around operator
2021-07-16 11:29:26,037 - stpipe - INFO - 34:26: E225 missing whitespace around operator


In [ ]:
def prepare_table(img, xy_tmp, aperture_radius, sky, flux, rad_label):
    '''
    Function that performs additional steps discussed in the notebook.
    To be used in the exercise.
    
    Inputs
    ----------
    img : ImageModel
        The data model of the image.
    xy_tmp : Table
        The table created by DAOStarFinder with the sources detected
        in the image.
    aperture_radius : float
        The aperture radius used for the aperture photometry.
    sky : array(float)
        The array with the local sky background for each source.
    flux : array(float)
        The array with the sky-subtracted flux for each source.
    rad_label : string
        The label for the photometry in the Table.
        
    Outputs
    ----------
    aperture_table : Table
        The final astro-photometric table.
    '''
    
    aperture_table = Table()

    aperture_table['x'] = xy_tmp['xcentroid']
    aperture_table['y'] = xy_tmp['ycentroid']
    aperture_table['sharpness'] = xy_tmp['sharpness']
    aperture_table['roundness1'] = xy_tmp['roundness1']
    aperture_table['roundness2'] = xy_tmp['roundness2']

    aperture_table['id'] = xy_tmp['id']
    
    aperture_table['local_sky_' + rad_label] = sky
    aperture_table['aperture_' + rad_label + '_skysub'] = flux
    
    keep_good = np.logical_and(np.isfinite(aperture_table['aperture_' + rad_label +'_skysub']), 
                               aperture_table['aperture_' + rad_label + '_skysub']>0.)
    aperture_table = aperture_table[keep_good]
        
    aperture_table['mag_' + rad_label] = -2.5*np.log10(aperture_table['aperture_' + rad_label + '_skysub'])

    aperture_table['flag_' + rad_label] = np.zeros(len(aperture_table), dtype=int)
    for s in aperture_table:
        jmin = max(1,int(np.floor(s['y']-aperture_radius)))
        jmax = min(round(s['y']+aperture_radius)+1,img.shape[0])
        imin = max(1,int(np.floor(s['x']-aperture_radius)))
        imax = min(round(s['x']+aperture_radius)+1,img.shape[1])
        if (np.sum(img.dq[jmin:jmax,imin:imax]==6)>0):
            s['flag_' + rad_label] = 6
        elif (np.sum(img.dq[jmin:jmax,imin:imax]==2)>0):
            s['flag_' + rad_label] = 2
        elif (np.sum(img.dq[jmin:jmax,imin:imax]==4)>0):
            s['flag_' + rad_label] = 4       

    aperture_table['ra'] = aperture_table['x']
    aperture_table['dec'] = aperture_table['y']
    for a in aperture_table:
        rd = img.meta.wcs.transform("detector", "world", a['x'], a['y'])    
        a['ra'] = rd[0]
        if (a['ra']>180):
            a['ra'] -= 360.0
        a['dec'] = rd[1]
        
    return aperture_table

2021-07-16 11:29:29,188 - stpipe - INFO - 41:84: E225 missing whitespace around operator
2021-07-16 11:29:29,188 - stpipe - INFO - 42:83: E225 missing whitespace around operator
2021-07-16 11:29:29,189 - stpipe - INFO - 49:21: E231 missing whitespace after ','
2021-07-16 11:29:29,189 - stpipe - INFO - 50:51: E231 missing whitespace after ','
2021-07-16 11:29:29,189 - stpipe - INFO - 51:21: E231 missing whitespace after ','
2021-07-16 11:29:29,190 - stpipe - INFO - 52:51: E231 missing whitespace after ','
2021-07-16 11:29:29,190 - stpipe - INFO - 53:36: E231 missing whitespace after ','
2021-07-16 11:29:29,190 - stpipe - INFO - 53:47: E225 missing whitespace around operator
2021-07-16 11:29:29,191 - stpipe - INFO - 53:51: E225 missing whitespace around operator
2021-07-16 11:29:29,191 - stpipe - INFO - 55:38: E231 missing whitespace after ','
2021-07-16 11:29:29,192 - stpipe - INFO - 55:49: E225 missing whitespace around operator
2021-07-16 11:29:29,192 - stpipe - INFO - 55:53: E225 mis

In [ ]:
def prepare_phot_cal(cat_name, tmp, sel_radius):
    '''
    Function that cross-matches two catalogs and plot the positional residuals.
    To be used in the exercise.
    
    Inputs
    ----------
    cat_name : string
        The name of the pipeline source catalog
    tmp : Table
        The astro-photometric table.
    sel_radius : float
        The radius for the positional-residual selection in arcsec.
        
    Outputs
    ----------
    calib_cat : Table
        The pipeline source catalog.
    ind_i2d_cat: array(int)
        The array with the indexes from match_sky_coordinates.
    dist_2d: array(float)
        The array with the 2D distances from match_sky_coordinates.
    '''
    
    calib_cat = Table.read(cat_name)

    coord_cal = SkyCoord(ra=tmp['ra'], dec=tmp['dec'], unit="deg")
    coord_i2d = SkyCoord(ra=calib_cat['sky_centroid'].ra.degree, dec=calib_cat['sky_centroid'].dec.degree, unit="deg")
    ind_i2d_cat, dist_2d, a = match_coordinates_sky(coord_cal, coord_i2d)

    delta_ra = 3600.0*(tmp['ra']*np.cos(np.deg2rad(tmp['dec']))-
                       calib_cat[ind_i2d_cat]['sky_centroid'].ra.degree*
                       np.cos(np.deg2rad(calib_cat[ind_i2d_cat]['sky_centroid'].dec.degree)))

    delta_dec = 3600.0*(tmp['dec']-calib_cat[ind_i2d_cat]['sky_centroid'].dec.degree)

    fig, ax = plt.subplots()
    ax.scatter(delta_ra, delta_dec, lw=0.5, s=5, color='black',marker='o')
    circle = plt.Circle((0, 0), sel_radius, color='r', fill=False)
    ax.add_patch(circle)
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)
    ax.axhline(0, color='red', ls='--')
    ax.axvline(0, color='red', ls='--')
    ax.set_aspect('equal', 'box')
    ax.set_xlabel(r'$\Delta\textrm{(R.A.}\cos\textrm{Dec.) [arcsec]}$')
    ax.set_ylabel(r'$\Delta\textrm{Dec. [arcsec]}$')

    ax2 = ax.secondary_xaxis('top', functions=(arcsec2pix, pix2arcsec))
    ax2.set_xlabel(r'$\Delta\textrm{(R.A.}\cos\textrm{Dec.) [MIRIM pixel]}$')
    ay2 = ax.secondary_yaxis('right', functions=(arcsec2pix, pix2arcsec))
    ay2.set_ylabel(r'$\Delta\textrm{Dec. [MIRIM pixel]}$')

    plt.tight_layout()

    return calib_cat, ind_i2d_cat, dist_2d

2021-07-16 11:29:31,648 - stpipe - INFO - 31:64: E225 missing whitespace around operator
2021-07-16 11:29:31,648 - stpipe - INFO - 32:72: E225 missing whitespace around operator
2021-07-16 11:29:31,648 - stpipe - INFO - 38:63: E231 missing whitespace after ','


### 2.4<font color='white'>-</font>Setup PSF FWHM<a class="anchor" id="fwhm"></a> ###

In [ ]:
filter_fwhm = {
    'F560W' : 1.636,
    'F770W' : 2.187
}

2021-07-16 11:29:34,513 - stpipe - INFO - 12: E203 whitespace before ':'
2021-07-16 11:29:34,514 - stpipe - INFO - 12: E203 whitespace before ':'


### 2.5<font color='white'>-</font>Download data<a class="anchor" id="download"></a> ###

Let's download and organize the data.

In [ ]:
boxlink = 'https://stsci.box.com/shared/static/8pjjn8nnaf1d1mev98ca9kc6mss7prmt.zip'
boxfile = './stage2-selected.zip'
urllib.request.urlretrieve(boxlink, boxfile)
with zipfile.ZipFile(boxfile) as zf:
    zf.extractall('./')

boxlink = 'https://stsci.box.com/shared/static/qstpome9vb95ay6aqlgvk0wvlku9ihay.zip'
boxfile = './stage3-selected.zip'
urllib.request.urlretrieve(boxlink, boxfile)
with zipfile.ZipFile(boxfile) as zf:
    zf.extractall('./')

In [ ]:
filter_names = ['F560W', 'F770W']
folder_path = './'

img_names = {}
cat_names = {}
for f in filter_names:
    print('')
    print(r'{0}-filter images:'.format(f))
    names = glob.glob('det_*' + f + '*_cal.fits')
    for n in range(len(names)):
        print(r'     {0}) {1}'.format(n+1,names[n]))
    img_names[f] = names
    names = glob.glob('complex_*' + f + '*cat*.ecsv')
    cat_names[f] = names
    print('')
    print(r'1 catalog found: {0}'.format(names[0]))

2021-07-16 11:31:48,946 - stpipe - INFO - 11:42: E231 missing whitespace after ','


## 3<font color='white'>-</font>A first glimpse to the MIRI data<a class="anchor" id="datalook"></a> ##

Let's have a look at a F560W-filter image. We start by reading our image as a [JWST data model](https://jwst-pipeline.readthedocs.io/en/latest/jwst/datamodels/) and displaying it. Refer to the JWebbinar 1 ([Pipeline Information and Data Products](https://www.stsci.edu/jwst/science-execution/jwebbinars)) material for a detailed description of the JWST data models.

In [ ]:
img_F560W = datamodels.open(img_names['F560W'][1])

First, we compute the $\sigma$-clipped median and sigma of the image. These values are then used to set the color scale for the image. Then, we use the function _imshow_me_wcolorbar_ to show the image. This function uses standard matplotlib tools. Note that pixel values are in mega Jansky per steradian (MJy sr$^{-1}$).

In [ ]:
_, med, sig = sigma_clipped_stats(img_F560W.data[np.isfinite(img_F560W.data)], sigma=5.0, maxiters=5)

# Inputs: 2D array, min value, max value, title, x-axis label, y-axis label, colorbar label, colormap
imshow_me_wcolorbar(img_F560W.data, med-1*sig, med+1*sig, 'Original image', 'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')

### 3.1<font color='white'>-</font>Data Quality (DQ) flags<a class="anchor" id="dqflag"></a> ###

Not all pixels should be used. We can use the Data Quality (DQ) flags to assess whether a pixel can be considered in calculations or not. [Here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/references_general/references_general.html#data-quality-flags) you can find a description of the DQ flags.

There is no one-size-fits-all solution for selecting pixels using the DQ flags. For this specific exercise, let's keep all pixels with DQ flag equal to:

- 0 = Good pixel
- 2 = Pixel saturated during integration
- 4 = Jump detected during integration
- 6 = Combination of DQ flags 2 and 4

As we can see, the DQ flags can correspond to multiple features. We can use _dqflags.dqflags_to_mnemonics_ to convert the DQ integer values into more user-friendly names:

In [ ]:
print(r' DQ flag equal to 2: {0}'.format(dqflags.dqflags_to_mnemonics(2, dqflags.group)))
print(r' DQ flag equal to 4: {0}'.format(dqflags.dqflags_to_mnemonics(4, dqflags.group)))
print(r' DQ flag equal to 6: {0}'.format(dqflags.dqflags_to_mnemonics(6, dqflags.group)))

Let's flag all pixels with a DQ flag different from these four values:

In [ ]:
ok = np.zeros(img_F560W.data.shape, dtype='int')
for v in [0, 2, 4, 6]:
    ok = ok + np.where(img_F560W.dq==v, 1, 0)

img_F560W.data[ok==0] = np.nan

print(r'{0} out of {1} pixels are not usable (~{2:3.1f}%)'.format((ok==0).sum(),
                                                                  img_F560W.data.shape[0]*img_F560W.data.shape[1],
                                                                  (ok==0).sum()/(img_F560W.data.shape[0]*img_F560W.data.shape[1])*100.0))

2021-07-16 11:52:22,980 - stpipe - INFO - 3:36: E225 missing whitespace around operator
2021-07-16 11:52:22,981 - stpipe - INFO - 5:18: E225 missing whitespace around operator
2021-07-16 11:52:22,981 - stpipe - INFO - 7:70: E225 missing whitespace around operator
2021-07-16 11:52:22,982 - stpipe - INFO - 9:70: E225 missing whitespace around operator


After masking, the same MIRI image looks different:

In [ ]:
imshow_me_wcolorbar(img_F560W.data, med-1*sig, med+1*sig, 'Image after masking', 'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')

As we can notice, the regions associated to the 4-quadrant phase mask (4QPM) coronagraphs disappeared from the image. The optical system of the 4QPM coronographs is different from that of the imager. Because of the way these optical elements affect the light transmission, the calibration of the 4QPM-coronograph regions is complicated and specific for these coronographs. Therefore, even though photons are detected in the regions of the coronagraphs during standard imaging observations, these regions should not be used while analyzing the Stage-2 image. If you run the _calwebb_image3_ pipeline (or the resample step in _calwebb_image2_), you will notice that the 4QPM regions are missing in the resampled image (_i2d.fits_) as well.

Another feature we can notice is that two columns (# 385 and 386) were flagged. Although a qualitative assessment of the image (for example with ds9) does not show anything particularly different from the other columns, these two columns showed to be coupled and should not be used for science.

## 4<font color='white'>-</font>From the image to the astro-photometric catalog<a class="anchor" id="work"></a> ##

There are various tools that can be used to perform aperture photometry on an image. In the following, we use [_photutils_](https://photutils.readthedocs.io/en/stable/index.html). Refer to the _photutils_ documentation for an extensive description of its tools.

### 4.1<font color='white'>-</font>Sky background<a class="anchor" id="sky"></a> ###

The JWST background is dominated by emissions from the zodiacal cloud of the Solar System, the Milky Way and the telescope itself. See the [official documentation](https://jwst-docs.stsci.edu/jwst-observatory-functionality/jwst-background-model) for a detailed discussion on this topic.

Sky background can be spatially variable in the image and subtracting the simple mean/median of all pixels would leave residuals in the image background. Thus, it is important to properly model and subtract the sky from each image. If dedicated background observations are available for your data set, you can use the pipeline _Background_ step. Otherwise, we can make a sky-background model from the image itself.

The _photutils_ python package provides various options for subtracting [2D background](https://photutils.readthedocs.io/en/stable/background.html). The basic idea is to divide the image in NxM subregions, estimate the background in each region, and finally create the low-resolution background image with a median filter. A detailed description is provided in the [_Background2D_](https://photutils.readthedocs.io/en/stable/api/photutils.background.Background2D.html#photutils.background.Background2D) documentation.

The choice of the input parameters for the _Background2D_ class is delicate. Some of the key input parameters to remember are the following:
- box_size : the size (in pixels) of the box in which to estimate the background;
- filter_size : the size of the window of the 2D median filter applied to the image to obtain the low-resolution background map;
- sigma_clip : the sigma-clipping parameters;
- bkg_estimator : the method used to compute the background;
- coverage_mask : the mask that tells if a pixel should be masked and not used in the computation.

These parameters should be fine tuned for every image according to some characteristics of the scene you are looking at, for example, the size of the sources, level of crowding, background gradient.

Let's create a 2D background model and subtract it from our image:

In [ ]:
# Mask all nan or inf pixels
mask = np.full(np.shape(img_F560W.data), False, dtype=bool)
mask[np.isnan(img_F560W.data)==True] = True
mask[np.isfinite(img_F560W.data)==False] = True

sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
# This is the background estimator -> DAOPHOT MMM algorithm - "mode" = 3*median - 2*mean
mmm_bkg = MMMBackground()
# Compute sky background
sky_F560W = Background2D(img_F560W.data, box_size=(20,20), filter_size=(30,30), 
                         sigma_clip=sigma_clip, bkg_estimator=mmm_bkg, coverage_mask=mask, fill_value=0.0)

print(r'Median background: {0}'.format(sky_F560W.background_median))
print(r'RMS background: {0}'.format(sky_F560W.background_rms_median))

img_F560W_skysub = img_F560W.data - sky_F560W.background

imshow_me_wcolorbar(img_F560W_skysub, -0.1, 0.25, 'Image sky subtracted',
                    'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

### 4.2<font color='white'>-</font>Aperture photometry<a class="anchor" id="phot"></a> ###

After we have subtracted the background, we can search for sources in the image using [_DAOStarFinder_](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder), an implementation of the DAOFIND algorithm (<a href="https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract">Stetson 1987, PASP 99, 191</a>).

_DAOStarFinder_ searches an image for local density maxima that have a peak amplitude greater than a specific threshold (applied to a convolved image) and have size and shape similar to a defined 2D Gaussian kernel. For this example, we search for objects at least 5$\times$ above the sky-background rms, and use as FWHM for the 2D Gaussian kernel the FWHM of the PSF in the F560W filter.

In [ ]:
# 5 times the background rms
threshold_F560W = 5.0*sky_F560W.background_rms_median

# Filter-dependent FWHM from the PSF
fwhm_F560W = filter_fwhm.get(img_F560W.meta.instrument.filter)

# Create DAOStarFinder instance
dsf_F560W = photutils.DAOStarFinder(threshold=threshold_F560W, fwhm=fwhm_F560W, exclude_border=True)

# Run DAOStarFinder on the subtracted image and save the output in a table
xy_F560W_tmp = dsf_F560W(img_F560W_skysub)

# Print 10 lines of the table
xy_F560W_tmp.pprint_all(max_lines=10)

The column description is almost straightforward. The three parameters that need further description are:
- sharpness : (height of the central pixel - mean of the surrounding non-bad pixels) / (height of the best fitting Gaussian function at that point);
- roundness1 : source symmetry;
- roundness2 : (difference in the height of the best fitting Gaussian function in x - the height of the the best fitting Gaussian function in y) / (average of the best fitting Gaussian functions in x and y).

These parameters can be used to discern between stars, galaxies or spurious detections. It is possible to setup _DAOStarFinder_ to exclude a priori objects outside a specific range of sharpness/roundness1/roundness2. Finally, note that the magnitude is defined as $-2.5\log_{10}(\text{peak density/detection threshold})$, which is just a rough estimate of the magnitude. An exaustive description of inputs and outputs is also provided [here](https://iraf.net/irafhelp.php?val=daofind).

Let's plot what we have found. We can use _imshow_me_wcolorbar_setup_, which is a function similar to _imshow_me_wcolorbar_ we used before but it allows us to keep plotting.

In [ ]:
fig, ax = plt.subplots()
imshow_me_wcolorbar_setup(img_F560W_skysub, -0.1, 0.25, 'Image sky subtracted', 'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')
ax.scatter(xy_F560W_tmp['xcentroid'], xy_F560W_tmp['ycentroid'], lw=0.5, s=15, marker='o', edgecolors='red', facecolors='none')
plt.tight_layout()

Now we can perform our aperture photometry using the [_aperture_photometry_](https://photutils.readthedocs.io/en/latest/aperture.html#aperture-photometry-photutils-aperture) function in _photutils_.

We need to store positions in a Nx2 array, with N being the number of stars found by _DAOStarFinder_. Then, we call _CircularAperture_, which defines a circular aperture for each star at position (x,y). We can define one or more aperture radii. For this example, let's use three radii based on the FWHM of the F560W-filter PSF:

In [ ]:
# Define the positions
positions_F560W = np.stack((xy_F560W_tmp['xcentroid'], xy_F560W_tmp['ycentroid']), axis=-1)

# Retrieve the aperture radii based on the FWHM of the F560W-filter PSF
r0 = filter_fwhm.get(img_F560W.meta.instrument.filter)
r1 = filter_fwhm.get(img_F560W.meta.instrument.filter)*2.0
r2 = filter_fwhm.get(img_F560W.meta.instrument.filter)*3.0
aper_radii = [r0, r1, r2]

print(r'Aperture radii used:')
print(r' r0 = {0:.3f} MIRIM pixel'.format(r0))
print(r' r1 = {0:.3f} MIRIM pixel'.format(r1))
print(r' r2 = {0:.3f} MIRIM pixel'.format(r2))
print('')

# Define the circular apertures
circular_apertures = [CircularAperture(positions_F560W, r=r) for r in aper_radii]

# Run the aperture photometry
phot_F560W_tmp = aperture_photometry(img_F560W.data*img_F560W.area, circular_apertures, method='exact')

# Print 10 lines
phot_F560W_tmp.pprint_all(max_lines=10)

The three fluxes are saved in aperture_sum_0, aperture_sum_1 and aperture_sum_2. Positions are exactly the same given in input, i.e., the output of _DAOStarFinder_.

<div class="alert alert-block alert-info">
    <h3><u><b>Note</b></u></h3>

As you can notice, the aperture photometry is performed on the image values multiplied by _img_F560W.area_.

The area subtended by each pixel on the sky varies as a result of geometric distortion and hence, they have different collecting power. Flat-fielding  is designed to produce images with a flat background given a flat stimulus. When the flat-field correction is applied to a science image, the resulting image is characterized by constant surface brightness. However, flat-fielding does not take into account for the different collecting power of pixels due to geometric distortion. For more details, see the discussion <a href="https://hst-docs.stsci.edu/wfc3dhb/chapter-9-wfc3-data-analysis/9-1-wfc3-data-analysis#id-9.1WFC3DataAnalysis-9.1.119.1.11PixelAreaMaps">here</a>.
    
Photometry measured on distorted images as the stage-2 _cal.fits_ files requires a field-dependent correction called the Pixel Area Map (PAM), to ensure that the same sources have the same magnitude regardless of their position within the images. The PAM is obtained from the geometric-distortion correction and is included in the _area_ extention of the JWST data model.

</ul>
</div>

These fluxes obviously include the sky background. Using a global value for the sky is not ideal, even if we work with a sky-subtracted image, because there can be unsubtracted background residuals left. Thus, we evaluate the local background for each source using a [circular annulus](https://photutils.readthedocs.io/en/latest/aperture.html#local-background-subtraction) of inner and outer radii of 25 and 35 MIRIM pixels, respectively. We can also mask pixels before evaluating the local background.

In [ ]:
# Define the annulus aperture
annulus_aperture = CircularAnnulus(positions_F560W, r_in=25.0, r_out=35.0)

# Define the mask with only pixels in each annulus
annulus_mask = annulus_aperture.to_mask(method='center')

# The local sky for each star will be stored in this list
local_sky_median = []

# For each source
for mask in annulus_mask:
    # Multiply the pixel values by the mask. Since the mask is either 0 or 1, 
    # the only non-zero pixels are those in the circular annulus
    annulus_data = mask.multiply(img_F560W.data*img_F560W.area)
    
    # Keep only non-masked pixels with finite values
    ok = np.logical_and(mask.data>0, np.isfinite(annulus_data))
    
    # If there are not at least 10 usable pixels in the annulus to compute
    # the local median sky, flag the star and remove it from the final list later
    if (np.sum(ok)>=10):
        # From 2D to 1D array
        annulus_data_1d = annulus_data[ok]
        # Sigma-clipped median
        _, median_sigclip, _ =  sigma_clipped_stats(annulus_data_1d, sigma=3.5, maxiters=5)
    else:
        # Flagged value
        median_sigclip = -99.99
    local_sky_median.append(median_sigclip)
    
# Convert list into array
local_sky_median_F560W = np.array(local_sky_median)

2021-07-16 12:05:21,713 - stpipe - INFO - 17:34: E225 missing whitespace around operator
2021-07-16 12:05:21,714 - stpipe - INFO - 21:19: E225 missing whitespace around operator
2021-07-16 12:05:21,714 - stpipe - INFO - 25:31: E222 multiple spaces after operator


The work is almost complete. We can make some order and save the final astro-photometric catalog.

In [ ]:
phot_F560W = Table()

# Useful info from DAOStarFinder
phot_F560W['x'] = xy_F560W_tmp['xcentroid']
phot_F560W['y'] = xy_F560W_tmp['ycentroid']
phot_F560W['sharpness'] = xy_F560W_tmp['sharpness']
phot_F560W['roundness1'] = xy_F560W_tmp['roundness1']
phot_F560W['roundness2'] = xy_F560W_tmp['roundness2']

# Save an ID
phot_F560W['id'] = xy_F560W_tmp['id']

# Local median sky
phot_F560W['annulus_median'] = local_sky_median_F560W

The total background within a circular aperture is the median local background times the area of the circular aperture. The area is saved as an attribute of the circular aperture.

In [ ]:
# Aperture photometry with r0
phot_F560W['local_sky_r0'] = local_sky_median_F560W*circular_apertures[0].area
phot_F560W['aperture_r0_skysub'] = phot_F560W_tmp['aperture_sum_0'] - phot_F560W['local_sky_r0']

# Aperture photometry with r1
phot_F560W['local_sky_r1'] = local_sky_median_F560W*circular_apertures[1].area
phot_F560W['aperture_r1_skysub'] = phot_F560W_tmp['aperture_sum_1'] - phot_F560W['local_sky_r1']

# Aperture photometry with r2
phot_F560W['local_sky_r2'] = local_sky_median_F560W*circular_apertures[2].area
phot_F560W['aperture_r2_skysub'] = phot_F560W_tmp['aperture_sum_2'] - phot_F560W['local_sky_r2']

# Print 10 lines
phot_F560W.pprint_all(max_lines=10)

Some stars are close to the edge of the MIRI imager and their flux cannot always be measured. Let's remove these stars:

In [ ]:
# Remove stars with flagged sky values
phot_F560W = phot_F560W[phot_F560W['annulus_median']>-90.0]

# Remove all stars that do not have a finite flux within r0, r1 and r2.
for r in [0, 1, 2]:
    keep_good = np.logical_and(np.isfinite(phot_F560W['aperture_r' + str(r) + '_skysub']),
                               phot_F560W['aperture_r' + str(r) + '_skysub']>0.)
    phot_F560W = phot_F560W[keep_good]

phot_F560W.pprint_all(max_lines=10)

2021-07-16 12:06:07,241 - stpipe - INFO - 2:53: E225 missing whitespace around operator
2021-07-16 12:06:07,241 - stpipe - INFO - 7:77: E225 missing whitespace around operator


We can compute the instrumental magnitude for each circular aperture:

In [ ]:
phot_F560W['mag_r0'] = -2.5*np.log10(phot_F560W['aperture_r0_skysub'])
phot_F560W['mag_r1'] = -2.5*np.log10(phot_F560W['aperture_r1_skysub'])
phot_F560W['mag_r2'] = -2.5*np.log10(phot_F560W['aperture_r2_skysub'])

At the beginning, we used the DQ flags to mask some pixels and kept only those with DQ$=$0, 2, 4, 6. Although still perfectly usable, you might want to keep track of pixels that saturated during an integration or were hit by a cosmic ray. For this reason, we define a flag by checking all pixels within each aperture radius we used:

In [ ]:
phot_F560W['flag_r0'] = np.zeros(len(phot_F560W), dtype=int)
phot_F560W['flag_r1'] = np.zeros(len(phot_F560W), dtype=int)
phot_F560W['flag_r2'] = np.zeros(len(phot_F560W), dtype=int)

for r, rl in zip([r0, r1, r2], ['r0', 'r1', 'r2']):
    for s in phot_F560W:
        jmin = max(1,int(np.floor(s['y']-r)))
        jmax = min(round(s['y']+r)+1,img_F560W.shape[0])
        imin = max(1,int(np.floor(s['x']-r)))
        imax = min(round(s['x']+r)+1,img_F560W.shape[1])
        if (np.sum(img_F560W.dq[jmin:jmax,imin:imax]==6)>0):
            s['flag_' + rl] = 6
        elif (np.sum(img_F560W.dq[jmin:jmax,imin:imax]==2)>0):
            s['flag_' + rl] = 2
        elif (np.sum(img_F560W.dq[jmin:jmax,imin:imax]==4)>0):
            s['flag_' + rl] = 4
            
print('                            Sources found: {0}'.format(len(phot_F560W)))
print('Sources with "SATURATED" pixels within r2: {0}'.format(np.sum(np.logical_or(phot_F560W['flag_r2']==2, phot_F560W['flag_r2']==6))))
print(' Sources with "JUMP_DET" pixels within r2: {0}'.format(np.sum(np.logical_or(phot_F560W['flag_r2']==4, phot_F560W['flag_r2']==6))))

2021-07-16 12:06:25,611 - stpipe - INFO - 7:21: E231 missing whitespace after ','
2021-07-16 12:06:25,611 - stpipe - INFO - 8:37: E231 missing whitespace after ','
2021-07-16 12:06:25,612 - stpipe - INFO - 9:21: E231 missing whitespace after ','
2021-07-16 12:06:25,612 - stpipe - INFO - 10:37: E231 missing whitespace after ','
2021-07-16 12:06:25,613 - stpipe - INFO - 11:42: E231 missing whitespace after ','
2021-07-16 12:06:25,613 - stpipe - INFO - 11:53: E225 missing whitespace around operator
2021-07-16 12:06:25,613 - stpipe - INFO - 11:57: E225 missing whitespace around operator
2021-07-16 12:06:25,614 - stpipe - INFO - 13:44: E231 missing whitespace after ','
2021-07-16 12:06:25,614 - stpipe - INFO - 13:55: E225 missing whitespace around operator
2021-07-16 12:06:25,615 - stpipe - INFO - 13:59: E225 missing whitespace around operator
2021-07-16 12:06:25,616 - stpipe - INFO - 15:44: E231 missing whitespace after ','
2021-07-16 12:06:25,616 - stpipe - INFO - 15:55: E225 missing whit

Let's check if our flag works. For example, let's test the jump-detection flag:

In [ ]:
# Flag for stars with "jump" pixel within r2 aperture
jump = np.logical_or(phot_F560W['flag_r2']==4, phot_F560W['flag_r2']==6)

fig, ax = plt.subplots()
imshow_me_wcolorbar_setup(img_F560W_skysub, -0.1, 0.25, 'Stars with "jump" pixel within r2 aperture', 'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')
ax.scatter(phot_F560W['x'], phot_F560W['y'], lw=0.5, s=15, marker='o', edgecolors='red', facecolors='none')
ax.scatter(phot_F560W['x'][jump], phot_F560W['y'][jump], c='deepskyblue', lw=0, s=20)
plt.tight_layout()

2021-07-16 12:06:40,684 - stpipe - INFO - 2:43: E225 missing whitespace around operator
2021-07-16 12:06:40,685 - stpipe - INFO - 2:69: E225 missing whitespace around operator


### 4.3<font color='white'>-</font>From pixel to equatorial coordinates<a class="anchor" id="wcs"></a> ###

We can transform the (x,y) positions from the raw, MIRIM coordinate system to the Equatorial system (ICRS) by means of the WCS information stored in the ASDF extension of our image.

<div class="alert alert-block alert-info">
    <h3><u><b>Nota bene</b></u></h3>
    
For stage-2 images (_cal.fits_ files), the WCS information is assigned by the _assign_wcs_ step. This information is saved in an ASDF extension of the FITS file. However, image-display tools such as ds9 do not understand the ASDF encoding. For this reason, an approximation to the WCS is stored in the image header using the SIP (Simple Imaging Polynomial) convention. The SIP-based header does not provide an exact fit: it is accurate to a $\sim$0.25-pixel level and it is meant for general display purposes (see <a href="https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/main.html">here</a> for more information).
</ul>
</div>

In [ ]:
phot_F560W['ra'] = phot_F560W['x']
phot_F560W['dec'] = phot_F560W['y']
for a in phot_F560W:
    rd = img_F560W.meta.wcs.transform("detector", "world", a['x'], a['y'])    
    a['ra'] = rd[0]
    if (a['ra']>180):
        a['ra'] -= 360.0
    a['dec'] = rd[1]

2021-07-16 12:07:07,333 - stpipe - INFO - 6:16: E225 missing whitespace around operator


### 4.4<font color='white'>-</font>Photometric calibration<a class="anchor" id="cal"></a> ###

The photometric calibration can be simply performed by cross-matching our photometric catalog with that generated by the _calwebb_image3_ pipeline.

In [ ]:
calib_cat_F560W = Table.read(cat_names['F560W'][0])
calib_cat_F560W.pprint_all(max_lines=1)

For the cross-match of these catalogs, we can use [_match_coordinates_sky_](https://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs). This function uses a proximity-based approach to find the same star in two catalogs. It requires coordinates to be stored as [_SkyCoord_](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord) objects.

In [ ]:
coord_cal = SkyCoord(ra=phot_F560W['ra'], dec=phot_F560W['dec'], unit="deg")
coord_i2d = SkyCoord(ra=calib_cat_F560W['sky_centroid'].ra.degree, dec=calib_cat_F560W['sky_centroid'].dec.degree, unit="deg")
ind_i2d_cat, dist_2d, _ = match_coordinates_sky(coord_cal, coord_i2d)

The first output _ind_i2d_cat_ is an array containing the indexes of the stars in the pipeline source catalog that are the closest objects to each source in our F560W-filter catalog. The second output _dist_2d_ is an array with the on-sky distances between sources. We do not need to save the last output because it corresponds to an array of 3D distances between sources. We do not need it here.

Being based just on proximity, this algorithm can present some issues in very crowded environments. It is always a good idea to check the result of the cross-match:

In [ ]:
delta_ra = 3600.0*(phot_F560W['ra']*np.cos(np.deg2rad(phot_F560W['dec']))-
                   calib_cat_F560W[ind_i2d_cat]['sky_centroid'].ra.degree*np.cos(np.deg2rad(calib_cat_F560W[ind_i2d_cat]['sky_centroid'].dec.degree)))

delta_dec = 3600.0*(phot_F560W['dec']-calib_cat_F560W[ind_i2d_cat]['sky_centroid'].dec.degree)

fig, ax = plt.subplots()
ax.scatter(delta_ra, delta_dec, lw=0.5, s=5, color='black',marker='o')
circle = plt.Circle((0, 0), 0.055, color='r', fill=False)
ax.add_patch(circle)
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.axhline(0, color='red', ls='--')
ax.axvline(0, color='red', ls='--')
ax.set_aspect('equal', 'box')
ax.set_xlabel(r'$\Delta\textrm{(R.A.}\cos\textrm{Dec.) [arcsec]}$')
ax.set_ylabel(r'$\Delta\textrm{Dec. [arcsec]}$')

ax2 = ax.secondary_xaxis('top', functions=(arcsec2pix, pix2arcsec))
ax2.set_xlabel(r'$\Delta\textrm{(R.A.}\cos\textrm{Dec.) [MIRIM pixel]}$')
ay2 = ax.secondary_yaxis('right', functions=(arcsec2pix, pix2arcsec))
ay2.set_ylabel(r'$\Delta\textrm{Dec. [MIRIM pixel]}$')

plt.tight_layout()

2021-07-16 12:07:31,283 - stpipe - INFO - 1:74: E225 missing whitespace around operator
2021-07-16 12:07:31,283 - stpipe - INFO - 7:59: E231 missing whitespace after ','


Let's define as "found" only objects with a positional rms lower than 0.5 MIRIM pixel, i.e., 0.055 arcsec.

In [ ]:
phot_F560W_matched = phot_F560W[dist_2d.arcsec/0.11<0.5]
calib_cat_F560W_matched = calib_cat_F560W[ind_i2d_cat[dist_2d.arcsec/0.11<0.5]]

2021-07-16 12:07:38,561 - stpipe - INFO - 1:52: E225 missing whitespace around operator
2021-07-16 12:07:38,561 - stpipe - INFO - 2:74: E225 missing whitespace around operator


Next, we need to compute the zero-point between the magnitudes in our custom _cal.fits_-based catalog and those in the pipeline source catalog. This simple step takes into account for (1) the finite-aperture correction and (2) the AB (or Vega) magnitude zero-point. The finite-aperture correction is necessary because the flux measured within a finite aperture is just a fraction of the total signal of a source. Let's plot the magnitude difference as function of our instrumental magnitude r1.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(phot_F560W_matched['mag_r1'], phot_F560W_matched['mag_r1']-calib_cat_F560W_matched['aper_total_abmag'], 
           lw=0.5, s=5, color='black',marker='o')
ax.set_xlim(-8, -2)
ax.set_ylim(-26, -24)
# Uncomment after next step
#ax.axhline(mag_med_F560W, color='red', ls='--')
ax.set_xlabel(r'F560W mag')
ax.set_ylabel(r'$\Delta$mag')
plt.tight_layout()

2021-07-16 12:07:51,648 - stpipe - INFO - 3:38: E231 missing whitespace after ','
2021-07-16 12:07:51,649 - stpipe - INFO - 7:1: E265 block comment should start with '# '


We do not need to use all stars. It is preferred to use only bright sources since they should be well-measured in both catalogs. For this specific example, we only apply a magnitude-based selection. You can apply all necessary criteria to ensure the selection of the best objects for the task.

In [ ]:
# Magnitude limit
ok = phot_F560W_matched['mag_r1']<-3

delta_mag = phot_F560W_matched['mag_r1'][ok]-calib_cat_F560W_matched['aper_total_abmag'][ok]

_, mag_med_F560W, mag_rms_F560W = sigma_clipped_stats(delta_mag, sigma=2.0, maxiters=10)

print('')
print(r' Magnitude zero-point: {0:.3f} mag'.format(mag_med_F560W))

2021-07-16 12:07:58,739 - stpipe - INFO - 2:34: E225 missing whitespace around operator


Finally, the calibrate AB mag of our stars is simply:

In [ ]:
phot_F560W['ABmag'] = phot_F560W['mag_r1'] - mag_med_F560W

The same procedure can be used to convert our instrumental magnitudes in the Vega system.

<div class="alert alert-block alert-info">
    <h3><u><b>Aperture photometry on stage-3 images</b></u></h3>
    
This notebook can be adapted to perform aperture photometry on stage-3 (_i2d.fits_) images. In this case, with a few expedients the photometric calibration does not need additional files.
    
First, we choose an aperture radius from the aperture-correction reference file available in the <a href="https://jwst-crds.stsci.edu/">JWST Calibration Reference Data System</a> (CRDS). For each filter (FILTER), array/subarray (PUPIL) and encircled-energy values (EEFRACTION), this file lists the aperture-correction values needed for correcting observed signals within a finite aperture to the estimated total signal for a source (APCORR), the aperture radius containing the EEFRACTION fraction of the total flux (RADIUS), the inner (SKYIN) and outer (SKYOUT) radii for the sky estimation. The APCORR value is a multiplicative correction that scales the measured flux to infinite aperture.

If we perform aperture photometry using these values of RADIUS, SKYIN and SKYOUT, then, once corrected the flux for the finite aperture, the photometric calibration is straighforward. Indeed, as for stage-2 images, the pixel unit in the stage-3 images is MJy sr$^{-1}$. This is an advantage for the photometric calibration because we can directly obtain magnitudes in the AB system (<a href="https://ui.adsabs.harvard.edu/abs/1964ApJ...140..689O/abstract">Oke 1964</a>; see also Sect. 7 of <a href="https://ui.adsabs.harvard.edu/abs/2005PASP..117.1049S/abstract">Sirianni et al. 2005</a>). The AB magnitude is defined as:

\begin{equation*}
    m_{\rm AB} = -2.5 \log_{10} f_\nu - 48.60
\end{equation*}

with $f_\nu$ in erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$, or as:

\begin{equation*}
    m_{\rm AB} = -2.5 \log_{10} f_\nu + 8.90
\end{equation*}

with $f_\nu$ in Jansky. Therefore, we can simply transform our total flux in MJy sr$^{-1}$ by multiplying it by $10^6$ and by the average pixel area in steradian. The header keyword _PIXAR_SR_ contains the average pixel area in steradian; the corresponding value in the JWST data-model scheme is included in _meta.photometry.pixelarea_steradians_. This value is added in the flux (photometric) calibrations performed by the _Photom_ step of the _calwebb_image2_ pipeline.

AB magnitudes can be converted to VEGA magnitudes by adding the AB-to-Vega magnitude offset available in the corresponding reference file in the CRDS.
    
<div class="alert alert-block alert-warning">
    <h3><u><b>Warning 1</b></u></h3>

The APCORR values include a correction for the sky-background subtraction obtained with the specific SKYIN and SKYOUT radii provided in the CRDS reference file. This correction takes into account the contribution of the PSF wings of the star to the local background. If different sky-background radii are used, the APCORR values cannot be used anymore. The best alternative option is to evaluate the sky background in an annulus far enough from the center of each star that the contribution of the PSF wings of the star to the local background is negligible, for example between 25 and 35 pixels as shown in this notebook. In this case, the total flux of a star can be obtained from the flux containging EEFRACTION of the total flux by multiplying it by EEFRACTION$^{-1}$.
</ul>
</div>
    
<div class="alert alert-block alert-warning">
    <h3><u><b>Warning 2</b></u></h3>

No pixel-area map correction is needed with stage-3 images.
</ul>
</div>

</ul>
</div>

## 5<font color='white'>-</font>Exercise<a class="anchor" id="exercise"></a> ##

Let's repeat some steps discussed before for another image in F770W filter. First, let's call this routine to read a new image. For this exercise, a random gradient has been added to the image so we are forced to find the new correct setup for _Background2D_. _This is not how a real image will look like_.

In [ ]:
img_F770W, img_F770W_mod, cmin, cmax = prepare_image(img_names['F770W'][1])

We can use these parameters as initial guesses for the colorbar.

In [ ]:
imshow_me_wcolorbar(img_F770W_mod, cmin, cmax, 'F770W-filter image', 'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')

The first step we need to do is to subtract the sky background. We are going to use the img_F770W_mod 2D array for this part of the exercise. Try to setup _Background2D_ with the correct parameters.

In [ ]:
mask = np.full(np.shape(img_F770W.data), False, dtype=bool)
mask[np.isfinite(img_F770W_mod)==False] = True

sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
mmm_bkg = MMMBackground()
# Complete the line below
sky_F770W = 

print(r'Median background: {0}'.format(sky_F770W.background_median))
print(r'RMS background: {0}'.format(sky_F770W.background_rms_median))

img_F770W_skysub = img_F770W_mod - sky_F770W.background

imshow_me_wcolorbar(img_F770W_skysub, -0.1, 0.1, 'F770W-filter sky-subtracted image', 'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')

SyntaxError: invalid syntax (<ipython-input-42-6b87b7acfaf7>, line 7)

ValueError: invalid literal for int() with base 10: " E712 comparison to False should be 'if cond is False"

Now we can use _DAOStarFinder_ to find sources in our image (keep using the img_F770W_mod 2D array).

In [ ]:
threshold_F770W = 5.0*sky_F770W.background_rms_median
fwhm_F770W = filter_fwhm.get(img_F770W.meta.instrument.filter)

dsf = photutils.DAOStarFinder(threshold=threshold_F770W, fwhm=fwhm_F770W, exclude_border=True)
xy_F770W_tmp = dsf(img_F770W_skysub)

xy_F770W_tmp.pprint_all(max_lines=10)

NameError: name 'sky_F770W' is not defined

Now perform aperture photometry on this image using an aperture radius equal to 2$\times$ the PSF FWHM in this filter. We can now use the original img_F770W data model for the remaining parts of the exercise.

In [ ]:
positions_F770W = np.stack((xy_F770W_tmp['xcentroid'], xy_F770W_tmp['ycentroid']), axis=-1)

r0 = filter_fwhm.get(img_F770W.meta.instrument.filter)*2.0
circular_apertures = CircularAperture(positions_F770W, r=r0)
phot_F770W_tmp = aperture_photometry(img_F770W.data*img_F770W.area, circular_apertures, method='exact')

annulus_aperture = CircularAnnulus(positions_F770W, r_in=25.0, r_out=35.0)
annulus_mask = annulus_aperture.to_mask(method='center')
local_sky_median = []
for mask in annulus_mask:
    annulus_data = mask.multiply(img_F770W.data*img_F770W.area)
    ok = np.logical_and(mask.data>0,np.isfinite(annulus_data))
    if (np.sum(ok)>=10):
        annulus_data_1d = annulus_data[ok]
        _, median_sigclip, _ =  sigma_clipped_stats(annulus_data_1d, sigma=3.5, maxiters=5)
    else:
        median_sigclip = -99.99
    local_sky_median.append(median_sigclip)
local_sky_median_F770W = np.array(local_sky_median)

# Complete the two lines below
sky_F770W = 
flux_F770W = 

2021-07-16 12:40:40,121 - stpipe - INFO - 12:34: E225 missing whitespace around operator
2021-07-16 12:40:40,122 - stpipe - INFO - 12:36: E231 missing whitespace after ','
2021-07-16 12:40:40,122 - stpipe - INFO - 13:19: E225 missing whitespace around operator
2021-07-16 12:40:40,122 - stpipe - INFO - 15:31: E222 multiple spaces after operator
2021-07-16 12:40:40,123 - stpipe - INFO - 12: E999 SyntaxError: invalid syntax


SyntaxError: invalid syntax (<ipython-input-44-0a72b9d6875d>, line 22)

The following function prepares a table similar to the photometric table created for the F560W-filter image.

In [ ]:
phot_F770W = prepare_table(img_F770W, xy_F770W_tmp, r0, sky_F770W, flux_F770W, 'r0')

phot_F770W.pprint_all(max_lines=10)

print('')
print('                            Sources found: {0}'.format(len(phot_F770W)))
print('Sources with "SATURATED" pixels within r0: {0}'.format(np.sum(np.logical_or(phot_F770W['flag_r0']==2, phot_F770W['flag_r0']==6))))
print(' Sources with "JUMP_DET" pixels within r0: {0}'.format(np.sum(np.logical_or(phot_F770W['flag_r0']==4, phot_F770W['flag_r0']==6))))

2021-07-16 12:40:44,999 - stpipe - INFO - 7:105: E225 missing whitespace around operator
2021-07-16 12:40:44,999 - stpipe - INFO - 7:131: E225 missing whitespace around operator
2021-07-16 12:40:45,000 - stpipe - INFO - 8:105: E225 missing whitespace around operator
2021-07-16 12:40:45,000 - stpipe - INFO - 8:131: E225 missing whitespace around operator


NameError: name 'xy_F770W_tmp' is not defined

Finally, calibrate the instrumental magnitudes using the pipeline source catalog.

In [ ]:
calib_cat_F770W, ind_i2d_cat, dist_2d = prepare_phot_cal(cat_names['F770W'][0], phot_F770W, 0.055)

NameError: name 'phot_F770W' is not defined

In [ ]:
# Complete the two lines below
phot_F770W_matched = 
calib_cat_F770W_matched = 

2021-07-16 12:40:52,338 - stpipe - INFO - 21: E999 SyntaxError: invalid syntax


SyntaxError: invalid syntax (<ipython-input-47-f0dc38efb711>, line 2)

Plot $\Delta$mag as a function of instrumental magnitude.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(phot_F770W_matched['mag_r0'], phot_F770W_matched['mag_r0']-calib_cat_F770W_matched['aper_total_abmag'],
           lw=0.5, s=5, color='black',marker='o')
ax.set_xlim(-8, -2)
ax.set_ylim(-26, -24)
# Uncomment after next step
#ax.axhline(mag_med_F770W, color='red', ls='--')
ax.set_xlabel(r'F770W mag')
ax.set_ylabel(r'$\Delta$mag')
plt.tight_layout()

2021-07-16 12:40:55,390 - stpipe - INFO - 3:38: E231 missing whitespace after ','
2021-07-16 12:40:55,391 - stpipe - INFO - 7:1: E265 block comment should start with '# '


NameError: name 'phot_F770W_matched' is not defined

In [ ]:
# Complete the line below
ok = 

delta_mag = phot_F770W_matched['mag_r0'][ok]-calib_cat_F770W_matched['aper_total_abmag'][ok]

# Complete the line below
_, mag_med_F770W, mag_rms_F770W = 

print('')
print(r' Magnitude zero-point: {0:.3f} mag'.format(mag_med_F770W))

phot_F770W['ABmag'] = phot_F770W['mag_r0'] - mag_med_F770W

2021-07-16 12:41:11,320 - stpipe - INFO - 5: E999 SyntaxError: invalid syntax


SyntaxError: invalid syntax (<ipython-input-49-9b60c0c8a0c1>, line 2)

Plot all sources found.

In [ ]:
fig, ax = plt.subplots()
imshow_me_wcolorbar_setup(img_F770W_skysub, -0.1, 0.25, 'Sourced found in this {0}-filter image'.format(img_F770W.meta.instrument.filter), 
                          'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')
ax.scatter(phot_F770W['x'], phot_F770W['y'], lw=0.5, s=15, marker='o', edgecolors='red', facecolors='none')
plt.tight_layout()

NameError: name 'img_F770W_skysub' is not defined

## 6<font color='white'>-</font>Bonus \#1: your first JWST mid-infrared color-magnitude diagram<a class="anchor" id="bonus1"></a> ##

We have built two catalogs for the same field in two filters. We can combine the photometry in these catalogs to create our first mid-infrared color-magnitude diagram.

In [ ]:
coord_F560W = SkyCoord(ra=phot_F560W['ra'], dec=phot_F560W['dec'], unit="deg")
coord_F770W = SkyCoord(ra=phot_F770W['ra'], dec=phot_F770W['dec'], unit="deg")
ind_F770W_cat, dist_2d, a = match_coordinates_sky(coord_F560W, coord_F770W)

NameError: name 'phot_F770W' is not defined

Let's check the result of the cross-match:

In [ ]:
delta_ra = 3600.0*(phot_F560W['ra']*np.cos(np.deg2rad(phot_F560W['dec']))-
                   phot_F770W['ra'][ind_F770W_cat]*np.cos(np.deg2rad(phot_F770W['dec'][ind_F770W_cat])))

delta_dec = 3600.0*(phot_F560W['dec']-phot_F770W['dec'][ind_F770W_cat])

fig, ax = plt.subplots()
ax.scatter(delta_ra, delta_dec, lw=0.5, s=5, color='black',marker='o')
circle = plt.Circle((0, 0), 0.055, color='r', fill=False)
ax.add_patch(circle)
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.axhline(0, color='red', ls='--')
ax.axvline(0, color='red', ls='--')
ax.set_aspect('equal', 'box')
ax.set_xlabel(r'$\Delta\textrm{(R.A.}\cos\textrm{Dec.) [arcsec]}$')
ax.set_ylabel(r'$\Delta\textrm{Dec. [arcsec]}$')

ax2 = ax.secondary_xaxis('top', functions=(arcsec2pix, pix2arcsec))
ax2.set_xlabel(r'$\Delta\textrm{(R.A.}\cos\textrm{Dec.) [MIRIM pixel]}$')
ay2 = ax.secondary_yaxis('right', functions=(arcsec2pix, pix2arcsec))
ay2.set_ylabel(r'$\Delta\textrm{Dec. [MIRIM pixel]}$')

plt.tight_layout()

2021-07-16 12:43:55,443 - stpipe - INFO - 1:74: E225 missing whitespace around operator
2021-07-16 12:43:55,444 - stpipe - INFO - 7:59: E231 missing whitespace after ','


NameError: name 'phot_F770W' is not defined

Let's define as "found" only objects with a positional rms lower than 0.5 MIRIM pixel, i.e., 0.055 arcsec:

In [ ]:
phot_F560W_matched = phot_F560W[dist_2d.arcsec/0.11<0.5]
phot_F770W_matched = phot_F770W[ind_F770W_cat[dist_2d.arcsec/0.11<0.5]]

2021-07-16 12:43:58,250 - stpipe - INFO - 1:52: E225 missing whitespace around operator
2021-07-16 12:43:58,250 - stpipe - INFO - 2:66: E225 missing whitespace around operator


NameError: name 'phot_F770W' is not defined

Finally, we can make our first mid-infrared color-magnitude diagram:

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1.75)
ax.scatter(phot_F560W_matched['ABmag']-phot_F770W_matched['ABmag'], 
           phot_F770W_matched['ABmag'], lw=0.5, s=5, marker='o', color='black')
ax.set_xlim(-01.0,0.0)
ax.set_ylim(24,17)
ax.set_xlabel(r'$(m_{\textrm{F560W}}-m_{\textrm{F770W}})$')
ax.set_ylabel(r'$m_{\textrm{F770W}}$')
ax.set_title(r'Your first Mid-Infrared CMD')
plt.tight_layout()

2021-07-16 12:44:00,722 - stpipe - INFO - 5:18: E231 missing whitespace after ','
2021-07-16 12:44:00,722 - stpipe - INFO - 6:15: E231 missing whitespace after ','


AttributeError: 'AxesSubplot' object has no attribute 'set_box_aspect'

## 7<font color='white'>-</font>Bonus \#2: concentration indexes<a class="anchor" id="bonus2"></a> ##

As we have seen in some plots above, our sample of detected sources is a mix of stars, galaxies and spurious detections. We can use some parameters provided by _DAOStarFinder_ to discern between these objects, but we can also compute our own diagnostic tools. For example, in analogy with what the JWST _calwebb_image3_ pipeline does, we can compute concentration indexes. Let's work with the F560W-filter catalog:

In [ ]:
phot_F560W['CI_1_0'] = phot_F560W['aperture_r1_skysub']/phot_F560W['aperture_r0_skysub']
phot_F560W['CI_2_0'] = phot_F560W['aperture_r2_skysub']/phot_F560W['aperture_r0_skysub']
phot_F560W['CI_2_1'] = phot_F560W['aperture_r2_skysub']/phot_F560W['aperture_r1_skysub']

Regardless of the total flux of a source, the ratio between fluxes measured within two different apertures should be different for stars, galaxies or spurious objects. For example, galaxies have likely a broader profile than stars. Therefore, we can use this piece of information to discern between these objects.

\[Do not uncomment all lines yet...\]

In [ ]:
fig, ax = plt.subplots(3,1)
ax[0].scatter(phot_F560W['ABmag'], phot_F560W['CI_1_0'], lw=0, s=5, marker='o', color='black')
# Uncomment this line after you run the next cell
#ax[0].scatter(phot_F560W['ABmag'][notok], phot_F560W['CI_1_0'][notok], lw=0, s=5, marker='o', color='red')
ax[0].set_xlim(17.9, 26)
ax[0].set_ylim(-1, 6)
ax[0].set_xlabel(r'$m_{\rm F560W}$')
ax[0].set_ylabel(r'CI\_1\_0')
ax[1].scatter(phot_F560W['ABmag'], phot_F560W['CI_2_1'], lw=0, s=5, marker='o', color='black')
# Uncomment this line after you run the next cell
#ax[1].scatter(phot_F560W['ABmag'][notok], phot_F560W['CI_2_1'][notok], lw=0, s=5, marker='o', color='red')
ax[1].set_xlim(17.9, 26)
ax[1].set_ylim(-1, 6)
ax[1].set_xlabel(r'$m_{\rm F560W}$')
ax[1].set_ylabel(r'CI\_2\_1')
ax[2].scatter(phot_F560W['CI_1_0'], phot_F560W['CI_2_1'], lw=0, s=5, marker='o', color='black')
# Uncomment this line after you run the next cell
#ax[2].scatter(phot_F560W['CI_1_0'][notok], phot_F560W['CI_2_1'][notok], lw=0, s=5, marker='o', color='red')
#ax[2].plot(xint, yint, color='red')
ax[2].set_xlim(0, 5.5)
ax[2].set_ylim(-1, 5.5)
ax[2].set_xlabel(r'CI\_1\_0')
ax[2].set_ylabel(r'CI\_2\_1')
plt.tight_layout()

2021-07-16 12:44:27,364 - stpipe - INFO - 1:25: E231 missing whitespace after ','
2021-07-16 12:44:27,365 - stpipe - INFO - 4:1: E265 block comment should start with '# '
2021-07-16 12:44:27,365 - stpipe - INFO - 11:1: E265 block comment should start with '# '
2021-07-16 12:44:27,366 - stpipe - INFO - 18:1: E265 block comment should start with '# '
2021-07-16 12:44:27,366 - stpipe - INFO - 19:1: E265 block comment should start with '# '


You can notice in the bottom panel that most points seem to cluster in a specific region. We can define a threshold to define two groups of objects ("ok" and "notok").

In [ ]:
xint = [ 0.6,  1.,  1.3,  1.7,  1.8, 2]
yint = [   0, 1.5,  1.8,  1.9,  1.4, 0]
ci_lim = np.interp(phot_F560W['CI_1_0'], xint, yint)

notok = phot_F560W['CI_2_1']>ci_lim
ok = phot_F560W['CI_2_1']<ci_lim

2021-07-16 12:47:06,765 - stpipe - INFO - 1:9: E201 whitespace after '['
2021-07-16 12:47:06,765 - stpipe - INFO - 2:9: E201 whitespace after '['
2021-07-16 12:47:06,766 - stpipe - INFO - 5:29: E225 missing whitespace around operator
2021-07-16 12:47:06,766 - stpipe - INFO - 6:26: E225 missing whitespace around operator


Let's go back to the previous plot and uncomment some lines.

We can now have a closer look at a few random objects that have been rejected using the concentration indexes. We can re-run this cell many times to see a variety of objects.

In [ ]:
id_sel = []
while (True):
    n = random.choice(phot_F560W[notok]['id'])
    if (n not in id_sel):
        id_sel.append(n)
    if (len(id_sel)==4):
        break
id_sel = np.array(id_sel)

imshow_cutouts(img_F560W_skysub, phot_F560W, id_sel, '$\Delta$X [MIRIM pixel]', '$\Delta$Y [MIRIM pixel]', 'binary')

2021-07-16 12:47:44,010 - stpipe - INFO - 6:20: E225 missing whitespace around operator


As we can see, our concentration-based selection is not perfect: sometimes we flag real faint stars as galaxies/spurious detections, sometimes we keep galaxies in our star sample. As in many cases, the choice of the best parameters for the purge is a tradeoff.

In [ ]:
fig, ax = plt.subplots()
imshow_me_wcolorbar_setup(img_F560W_skysub, -0.1, 0.25, 'Concentration-index selection', 
                          'x [MIRIM pixel]', 'y [MIRIM pixel]', 'MJy sr$^{-1}$', 'binary')
ax.scatter(phot_F560W['x'][ok], phot_F560W['y'][ok], lw=0.5, s=15, marker='o', edgecolors='red', facecolors='none')
ax.scatter(phot_F560W['x'][notok], phot_F560W['y'][notok], lw=0.5, s=15, marker='o', edgecolors='deepskyblue', facecolors='none')
plt.tight_layout()

We have done! Here there is a bonus plot!

In [ ]:
fig, ax = plt.subplots()
fmin = np.min(phot_F560W['aperture_r2_skysub'])
fmax = np.max(phot_F560W['aperture_r2_skysub'])
size = 1.e4*(phot_F560W['aperture_r2_skysub']-fmin)/(fmax-fmin)
ax.scatter(phot_F560W['ra'][notok], phot_F560W['dec'][notok], s=size[notok], alpha=0.25, marker='.', color='deepskyblue')
ax.scatter(phot_F560W['ra'][ok], phot_F560W['dec'][ok], s=size[ok], alpha=0.25, marker='.', color='red')
ax.set_xlim(0.025, -0.013)
ax.set_ylim(-0.0215, 0.0165)
ax.set_xlabel(r'R.A. [deg]')
ax.set_ylabel(r'Dec. [deg]')
ax.text(0.3, 1.015, 'OK sources', verticalalignment='center', horizontalalignment='center', transform=ax.transAxes, color='red')
ax.text(0.7, 1.015, 'not-OK sources', verticalalignment='center', horizontalalignment='center', transform=ax.transAxes, color='deepskyblue')
plt.tight_layout()

## 8<font color='white'>-</font>Exercise solutions<a class="anchor" id="exercisesol"></a> ##

Missing line #1:

    sky_F770W = Background2D(img_F770W_mod, box_size=(20,20), filter_size=(10,10), sigma_clip=sigma_clip, bkg_estimator=mmm_bkg, coverage_mask=mask, fill_value=0.0)
    
    (Note that box_size and filter_size parameters might need to be fine-tuned for each exercise)

Missing lines #2 and #3:

    sky_F770W = local_sky_median_F770W*circular_apertures.area
    flux_F770W = phot_F770W_tmp['aperture_sum'] - sky_F770W

Missing lines #4 and #5:

    phot_F770W_matched = phot_F770W[dist_2d.arcsec/0.11<0.5]
    calib_cat_F770W_matched = calib_cat_F770W[ind_i2d_cat[dist_2d.arcsec/0.11<0.5]]

Missing line #6:
    
    ok = phot_F770W_matched['mag_r0']<-3.5

Missing line #7:

    _, mag_med_F770W, mag_rms_F770W = sigma_clipped_stats(delta_mag, sigma=2.0, maxiters=10)

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>